<a href="https://colab.research.google.com/github/myniggname/abednigo.github.io/blob/main/Application_Roll_Out_Adoption_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#allow access on google drive

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
#importing required library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
#standardizing personal font & figure

plt.style.use(style = 'tableau-colorblind10')
plt.rcParams['figure.figsize'] = (15, 10)
sns.set(rc={'figure.figsize':(25,20)})

In [ ]:
#importing dataset from bigquery

%%bigquery --project revoufsda-350705 event_flow

SELECT
event_name,
COUNT(event_id) event_flow, 
FROM `revoufsda-350705.Ecommerce.Event` event
GROUP BY event_name
ORDER BY event_flow DESC

# where transaction.status = 'Completed' 

In [ ]:
event_flow

,event_name,event_flow
0,Product view,18036
1,Add to cart,15501
2,View cart,14645
3,Using promo code,13400
4,Choose address,12377
5,Choose delivery services,11819
6,Change payment method,11311
7,Checkout,10493
8,Completed,9549
9,Cancelled,944


In [ ]:
#importing dataset from bigquery

%%bigquery --project revoufsda-350705 count_event

WITH event_flag AS
(
  SELECT
  event_name,
  favorite_flag,
  COUNT(event_id) event_flow, 
  FROM `revoufsda-350705.Ecommerce.Event` event
  GROUP BY event_name, favorite_flag
  ORDER BY 1, 2 DESC
),
event_flag_0 AS
(
  SELECT
  event_name,
  favorite_flag,
  event_flow AS unused_feature
  FROM event_flag
  WHERE favorite_flag = "0"
  ORDER BY event_flow DESC
),
event_flag_1 AS
(
  SELECT
  event_name,
  favorite_flag,
  event_flow AS used_feature
  FROM event_flag
  WHERE favorite_flag = "1"
  ORDER BY event_flow DESC
),
event_flag_null AS
(
  SELECT
  event_name,
  favorite_flag,
  event_flow AS uncount_feature
  FROM event_flag
  WHERE favorite_flag IS NULL
  ORDER BY event_flow DESC
)
SELECT
a.event_name,
a.unused_feature,
b.used_feature,
c.uncount_feature
FROM event_flag_0 a
FULL JOIN event_flag_1 b
ON a.event_name = b.event_name
FULL JOIN event_flag_null c
ON b.event_name = c.event_name
ORDER BY b.used_feature DESC


In [ ]:
count_event

,event_name,unused_feature,used_feature,uncount_feature
0,Product view,3744.0,8645.0,5647.0
1,Add to cart,3291.0,7627.0,4583.0
2,View cart,3049.0,7016.0,4580.0
3,Using promo code,2851.0,6500.0,4049.0
4,Choose address,2567.0,6344.0,3466.0
5,Choose delivery services,2316.0,6137.0,3366.0
6,Change payment method,2084.0,5972.0,3255.0
7,Checkout,1942.0,5639.0,2912.0
8,Completed,1724.0,5086.0,2739.0
9,Cancelled,218.0,553.0,173.0


In [ ]:
count_event

,event_name,unused_feature
0,Product view,3744.0
1,Add to cart,3291.0
2,View cart,3049.0
3,Using promo code,2851.0
4,Choose address,2567.0
5,Choose delivery services,2316.0
6,Change payment method,2084.0
7,Checkout,1942.0
8,Completed,1724.0
9,Cancelled,218.0


In [ ]:
last_year_event = count_event[{'event_name',
             'uncount_feature'}]
last_year_event

,uncount_feature,event_name
0,5647.0,Product view
1,4583.0,Add to cart
2,4580.0,View cart
3,4049.0,Using promo code
4,3466.0,Choose address
5,3366.0,Choose delivery services
6,3255.0,Change payment method
7,2912.0,Checkout
8,2739.0,Completed
9,173.0,Cancelled


In [ ]:
this_year_event_unused = count_event.drop(columns = count_event.columns[2:].tolist(), inplace=False)
this_year_event_unused

# df_clean_col = df.drop(columns=df.columns[6:].tolist(),inplace=False)

,event_name,unused_feature
0,Product view,3744.0
1,Add to cart,3291.0
2,View cart,3049.0
3,Using promo code,2851.0
4,Choose address,2567.0
5,Choose delivery services,2316.0
6,Change payment method,2084.0
7,Checkout,1942.0
8,Completed,1724.0
9,Cancelled,218.0


In [ ]:
this_year_event_used = count_event.drop(columns = count_event.columns[4:].tolist(), inplace=False)
this_year_event_used

,event_name,unused_feature
0,Product view,3744.0
1,Add to cart,3291.0
2,View cart,3049.0
3,Using promo code,2851.0
4,Choose address,2567.0
5,Choose delivery services,2316.0
6,Change payment method,2084.0
7,Checkout,1942.0
8,Completed,1724.0
9,Cancelled,218.0
